<a href="https://colab.research.google.com/github/harrymkwn/InfluenceAnalysis/blob/master/LSTMEmoInt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
from google.colab import  drive
drive.mount('/content/drive')
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
device_name = tf.test.gpu_device_name()
print(device_name)
df = pd.read_csv('/content/drive/My Drive/IA/Clean_Tweets_data.csv')
def remove_stop(text):
  sw = stopwords.words("english")
  sw.append('br')
  clean = " ".join([x for x in text.split() if x not in sw])
  return clean
df['Tweet'] = df['Tweet'].apply(lambda tweet : remove_stop(tweet))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/device:GPU:0


In [2]:
category_dict = {"anger":[1,0,0,0],"sadness":[0,1,0,0],"fear":[0,0,1,0],"joy":[0,0,0,1]}
data_tweet = [x.lower().split() for x in df['Tweet']]
data_cat = np.array([category_dict[x] for x in df['Emotion']])

In [3]:
print(data_tweet[:5])
print(data_cat[:5])

[['like', 'i', 'cannot', '12', 'hours', 'worth', 'standing', 'calorie', 'burning', '3', 'hours', 'loudly', 'crying', 'faceloudly', 'crying', 'faceloudly', 'crying', 'faceloudly', 'crying', 'face', '1stworldprobs4vs'], ['darkest', 'nights', 'bright', 'solemn', 'faces', 'lights', 'smile', 'believe', 'i'], ['true', 'depression', 'fans', 'get', 'one', 'face', 'tears', 'joyface', 'tears', 'joy'], ['awe', 'yay', 'thank', 'good', 'i', 'worried'], ['on', 'way', 'rejoicing', 'pulpit', 'park', 'city', 'baptist', 'church', 'surrey', 'bc', 'midweek', 'service']]
[[1 0 0 0]
 [0 1 0 0]
 [0 1 0 0]
 [0 0 1 0]
 [0 0 0 1]]


In [4]:
# Parameters
Min_count = 0
Embedding_size = 100
Window_size = 5
Negative_sampling = 00

In [5]:
w2v_sg = Word2Vec(min_count=Min_count,
                     window=Window_size,
                     size=Embedding_size,
                     negative=Negative_sampling,sg=1)
w2v_sg.build_vocab(data_tweet)
w2v_sg.train(data_tweet, total_examples=w2v_sg.corpus_count, epochs=5)
w2v_cbow = Word2Vec(min_count=Min_count,
                     window=Window_size,
                     size=Embedding_size,
                     negative=Negative_sampling,sg=0)
w2v_cbow.build_vocab(data_tweet)
w2v_cbow.train(data_tweet, total_examples=w2v_cbow.corpus_count, epochs=5)

(350729, 369570)

In [6]:
vocab_sg = w2v_sg.wv.vocab
vocab_sg = [x for x in vocab_sg]
vocab_cbow = w2v_cbow.wv.vocab
vocab_cbow = [x for x in vocab_cbow] 
vocab_glove = {}
with open("/content/drive/My Drive/InfluenceAnalysis/glove/glove.twitter.27B.100d.txt", 'r', encoding="utf-8") as f:
  for line in f:
      values = line.split()
      word = values[0]
      vector = np.asarray(values[1:], "float32")
      vocab_glove[word] = vector

In [7]:
data_tweet_vec_sg = np.array([np.array([w2v_sg.wv.get_vector(i) for i in x if i in vocab_sg]) for x in data_tweet])
data_tweet_vec_cbow = np.array([np.array([w2v_cbow.wv.get_vector(i) for i in x if i in vocab_cbow]) for x in data_tweet])
data_tweet_vec_glove = np.array([np.array([vocab_glove[i] for i in x if i in vocab_glove.keys()]) for x in data_tweet])
Max_input_size_glove = max([len(x) for x in data_tweet_vec_glove])
Max_input_size=max([len(x) for x in data_tweet_vec_sg]+[len(x) for x in data_tweet_vec_cbow])

In [8]:
data_tweet_vec_sg = np.array([np.pad(x.flatten(),(0,Max_input_size*Embedding_size-len(x.flatten()))).reshape(Max_input_size,Embedding_size) for x in data_tweet_vec_sg])
data_tweet_vec_cbow = np.array([np.pad(x.flatten(),(0,Max_input_size*Embedding_size-len(x.flatten()))).reshape(Max_input_size,Embedding_size) for x in data_tweet_vec_cbow])
data_tweet_vec_glove = np.array([np.pad(x.flatten(),(0,Max_input_size_glove*100-len(x.flatten()))).reshape(Max_input_size_glove,100) for x in data_tweet_vec_glove])


In [9]:
X_train_sg, X_test_sg, y_train_sg, y_test_sg = train_test_split(data_tweet_vec_sg, data_cat, test_size=0.30, random_state=42)
X_train_cbow, X_test_cbow, y_train_cbow, y_test_cbow = train_test_split(data_tweet_vec_cbow, data_cat, test_size=0.30, random_state=42)
X_train_glove, X_test_glove, y_train_glove, y_test_glove = train_test_split(data_tweet_vec_glove, data_cat, test_size=0.30, random_state=42)

In [10]:
model_sg = tf.keras.Sequential()
model_sg.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_sg.add(tf.keras.layers.LSTM(50))
model_sg.add(tf.keras.layers.Dense(4, activation='softmax'))
model_sg.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_sg.fit(X_train_sg,y_train_sg,epochs=20,validation_split=0.2)
history = model_sg.evaluate(X_test_sg,y_test_sg)
print(history)
print("SG_LSTM")

Epoch 1/20
125/125 [==============================] - 1s 9ms/step - loss: 1.3723 - accuracy: 0.3262 - val_loss: 1.3946 - val_accuracy: 0.2978
Epoch 2/20
125/125 [==============================] - 1s 6ms/step - loss: 1.3714 - accuracy: 0.3272 - val_loss: 1.3821 - val_accuracy: 0.2978
Epoch 3/20
125/125 [==============================] - 1s 6ms/step - loss: 1.3708 - accuracy: 0.3272 - val_loss: 1.3834 - val_accuracy: 0.2978
Epoch 4/20
125/125 [==============================] - 1s 6ms/step - loss: 1.3706 - accuracy: 0.3272 - val_loss: 1.3845 - val_accuracy: 0.2978
Epoch 5/20
125/125 [==============================] - 1s 5ms/step - loss: 1.3700 - accuracy: 0.3272 - val_loss: 1.3830 - val_accuracy: 0.2978
Epoch 6/20
125/125 [==============================] - 1s 5ms/step - loss: 1.3703 - accuracy: 0.3272 - val_loss: 1.3847 - val_accuracy: 0.2978
Epoch 7/20
125/125 [==============================] - 1s 6ms/step - loss: 1.3702 - accuracy: 0.3272 - val_loss: 1.3852 - val_accuracy: 0.2978
Epoch 

In [17]:
model_sg_bi = tf.keras.Sequential()
model_sg_bi.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_sg_bi.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50),merge_mode='concat'))
model_sg_bi.add(tf.keras.layers.Dense(4, activation='softmax'))
model_sg_bi.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_sg_bi.fit(X_train_sg,y_train_sg,epochs=20,validation_split=0.2)
history = model_sg_bi.evaluate(X_test_sg,y_test_sg)
print(history)
print("SG_BI")

Epoch 1/20
125/125 [==============================] - 2s 13ms/step - loss: 1.3734 - accuracy: 0.3194 - val_loss: 1.3848 - val_accuracy: 0.2978
Epoch 2/20
125/125 [==============================] - 1s 9ms/step - loss: 1.3701 - accuracy: 0.3272 - val_loss: 1.3825 - val_accuracy: 0.2978
Epoch 3/20
125/125 [==============================] - 1s 9ms/step - loss: 1.3699 - accuracy: 0.3272 - val_loss: 1.3832 - val_accuracy: 0.2978
Epoch 4/20
125/125 [==============================] - 1s 9ms/step - loss: 1.3697 - accuracy: 0.3272 - val_loss: 1.3843 - val_accuracy: 0.2978
Epoch 5/20
125/125 [==============================] - 1s 9ms/step - loss: 1.3692 - accuracy: 0.3272 - val_loss: 1.3818 - val_accuracy: 0.2978
Epoch 6/20
125/125 [==============================] - 1s 8ms/step - loss: 1.3689 - accuracy: 0.3272 - val_loss: 1.3844 - val_accuracy: 0.2978
Epoch 7/20
125/125 [==============================] - 1s 9ms/step - loss: 1.3671 - accuracy: 0.3272 - val_loss: 1.3809 - val_accuracy: 0.2978
Epoch

In [11]:
model_sg_rnn = tf.keras.Sequential()
model_sg_rnn.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_sg_rnn.add(tf.keras.layers.SimpleRNN(50))
model_sg_rnn.add(tf.keras.layers.Dense(4, activation='softmax'))
model_sg_rnn.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_sg_rnn.fit(X_train_sg,y_train_sg,epochs=20,validation_split=0.2)
history = model_sg_rnn.evaluate(X_test_sg,y_test_sg)
print(history)
print("SG_RNN")

Epoch 1/20
125/125 [==============================] - 4s 34ms/step - loss: 1.3758 - accuracy: 0.3219 - val_loss: 1.3807 - val_accuracy: 0.2978
Epoch 2/20
125/125 [==============================] - 4s 31ms/step - loss: 1.3684 - accuracy: 0.3265 - val_loss: 1.3806 - val_accuracy: 0.2988
Epoch 3/20
125/125 [==============================] - 4s 31ms/step - loss: 1.3693 - accuracy: 0.3262 - val_loss: 1.3840 - val_accuracy: 0.2948
Epoch 4/20
125/125 [==============================] - 4s 31ms/step - loss: 1.3679 - accuracy: 0.3242 - val_loss: 1.3851 - val_accuracy: 0.2978
Epoch 5/20
125/125 [==============================] - 4s 30ms/step - loss: 1.3665 - accuracy: 0.3300 - val_loss: 1.3880 - val_accuracy: 0.2978
Epoch 6/20
125/125 [==============================] - 4s 32ms/step - loss: 1.3663 - accuracy: 0.3262 - val_loss: 1.4082 - val_accuracy: 0.2777
Epoch 7/20
125/125 [==============================] - 4s 32ms/step - loss: 1.3691 - accuracy: 0.3285 - val_loss: 1.3840 - val_accuracy: 0.2978

In [12]:
model_cbow = tf.keras.Sequential()
model_cbow.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_cbow.add(tf.keras.layers.LSTM(50))
model_cbow.add(tf.keras.layers.Dense(4, activation='softmax'))
model_cbow.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_cbow.fit(X_train_cbow,y_train_cbow,epochs=20,validation_split=0.2)
history = model_cbow.evaluate(X_test_cbow,y_test_cbow)
print(history)
print("CBOW_LSTM")

Epoch 1/20
125/125 [==============================] - 1s 8ms/step - loss: 1.3745 - accuracy: 0.3179 - val_loss: 1.3827 - val_accuracy: 0.2978
Epoch 2/20
125/125 [==============================] - 1s 6ms/step - loss: 1.3701 - accuracy: 0.3272 - val_loss: 1.3829 - val_accuracy: 0.2978
Epoch 3/20
125/125 [==============================] - 1s 6ms/step - loss: 1.3708 - accuracy: 0.3272 - val_loss: 1.3828 - val_accuracy: 0.2978
Epoch 4/20
125/125 [==============================] - 1s 6ms/step - loss: 1.3707 - accuracy: 0.3272 - val_loss: 1.3850 - val_accuracy: 0.2978
Epoch 5/20
125/125 [==============================] - 1s 6ms/step - loss: 1.3698 - accuracy: 0.3272 - val_loss: 1.3821 - val_accuracy: 0.2978
Epoch 6/20
125/125 [==============================] - 1s 6ms/step - loss: 1.3706 - accuracy: 0.3272 - val_loss: 1.3872 - val_accuracy: 0.2978
Epoch 7/20
125/125 [==============================] - 1s 6ms/step - loss: 1.3701 - accuracy: 0.3272 - val_loss: 1.3828 - val_accuracy: 0.2978
Epoch 

In [18]:
model_cbow_bi = tf.keras.Sequential()
model_cbow_bi.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_cbow_bi.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50),merge_mode='concat'))
model_cbow_bi.add(tf.keras.layers.Dense(4, activation='softmax'))
model_cbow_bi.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_cbow_bi.fit(X_train_cbow,y_train_cbow,epochs=20,validation_split=0.2)
history = model_cbow_bi.evaluate(X_test_cbow,y_test_cbow)
print(history)
print("CBOW_BI")

Epoch 1/20
125/125 [==============================] - 2s 13ms/step - loss: 1.3726 - accuracy: 0.3234 - val_loss: 1.3839 - val_accuracy: 0.2978
Epoch 2/20
125/125 [==============================] - 1s 10ms/step - loss: 1.3709 - accuracy: 0.3272 - val_loss: 1.3844 - val_accuracy: 0.2978
Epoch 3/20
125/125 [==============================] - 1s 9ms/step - loss: 1.3699 - accuracy: 0.3272 - val_loss: 1.3876 - val_accuracy: 0.2978
Epoch 4/20
125/125 [==============================] - 1s 9ms/step - loss: 1.3702 - accuracy: 0.3272 - val_loss: 1.3848 - val_accuracy: 0.2978
Epoch 5/20
125/125 [==============================] - 1s 9ms/step - loss: 1.3691 - accuracy: 0.3272 - val_loss: 1.3849 - val_accuracy: 0.2978
Epoch 6/20
125/125 [==============================] - 1s 9ms/step - loss: 1.3686 - accuracy: 0.3272 - val_loss: 1.3833 - val_accuracy: 0.2978
Epoch 7/20
125/125 [==============================] - 1s 9ms/step - loss: 1.3677 - accuracy: 0.3272 - val_loss: 1.3846 - val_accuracy: 0.2978
Epoc

In [13]:
model_cbow_rnn = tf.keras.Sequential()
model_cbow_rnn.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_cbow_rnn.add(tf.keras.layers.SimpleRNN(50))
model_cbow_rnn.add(tf.keras.layers.Dense(4, activation='softmax'))
model_cbow_rnn.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_cbow_rnn.fit(X_train_cbow,y_train_cbow,epochs=20,validation_split=0.2)
history = model_cbow_rnn.evaluate(X_test_cbow,y_test_cbow)
print(history)
print("CBOW_RNN")

Epoch 1/20
125/125 [==============================] - 4s 32ms/step - loss: 1.3758 - accuracy: 0.3192 - val_loss: 1.3834 - val_accuracy: 0.2978
Epoch 2/20
125/125 [==============================] - 4s 33ms/step - loss: 1.3691 - accuracy: 0.3222 - val_loss: 1.3846 - val_accuracy: 0.2978
Epoch 3/20
125/125 [==============================] - 4s 31ms/step - loss: 1.3679 - accuracy: 0.3272 - val_loss: 1.3858 - val_accuracy: 0.2988
Epoch 4/20
125/125 [==============================] - 4s 31ms/step - loss: 1.3670 - accuracy: 0.3265 - val_loss: 1.3815 - val_accuracy: 0.2998
Epoch 5/20
125/125 [==============================] - 4s 30ms/step - loss: 1.3623 - accuracy: 0.3322 - val_loss: 1.4025 - val_accuracy: 0.3038
Epoch 6/20
125/125 [==============================] - 4s 30ms/step - loss: 1.3642 - accuracy: 0.3300 - val_loss: 1.3824 - val_accuracy: 0.2998
Epoch 7/20
125/125 [==============================] - 4s 30ms/step - loss: 1.3634 - accuracy: 0.3199 - val_loss: 1.3819 - val_accuracy: 0.3018

In [14]:
model_glove = tf.keras.Sequential()
model_glove.add(tf.keras.Input(shape=(Max_input_size_glove,100)))
model_glove.add(tf.keras.layers.LSTM(50))
model_glove.add(tf.keras.layers.Dense(4, activation='softmax'))
model_glove.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_glove.fit(X_train_glove,y_train_glove,epochs=20,validation_split=0.2)
history = model_glove.evaluate(X_test_glove,y_test_glove)
print(history)
print("GLOVE_LSTM")

Epoch 1/20
125/125 [==============================] - 1s 8ms/step - loss: 1.2846 - accuracy: 0.4088 - val_loss: 1.2409 - val_accuracy: 0.4245
Epoch 2/20
125/125 [==============================] - 1s 5ms/step - loss: 1.1752 - accuracy: 0.4719 - val_loss: 1.2153 - val_accuracy: 0.4256
Epoch 3/20
125/125 [==============================] - 1s 5ms/step - loss: 1.1139 - accuracy: 0.4873 - val_loss: 1.1992 - val_accuracy: 0.4396
Epoch 4/20
125/125 [==============================] - 1s 5ms/step - loss: 1.0751 - accuracy: 0.5049 - val_loss: 1.1335 - val_accuracy: 0.4497
Epoch 5/20
125/125 [==============================] - 1s 5ms/step - loss: 1.0191 - accuracy: 0.5243 - val_loss: 1.0479 - val_accuracy: 0.4960
Epoch 6/20
125/125 [==============================] - 1s 6ms/step - loss: 0.9626 - accuracy: 0.5588 - val_loss: 1.0152 - val_accuracy: 0.5221
Epoch 7/20
125/125 [==============================] - 1s 6ms/step - loss: 0.8879 - accuracy: 0.5995 - val_loss: 1.0043 - val_accuracy: 0.5664
Epoch 

In [19]:
model_glove_bi = tf.keras.Sequential()
model_glove_bi.add(tf.keras.Input(shape=(Max_input_size_glove,100)))
model_glove_bi.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50),merge_mode='concat'))
model_glove_bi.add(tf.keras.layers.Dense(4, activation='softmax'))
model_glove_bi.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_glove_bi.fit(X_train_glove,y_train_glove,epochs=20,validation_split=0.2)
history = model_glove_bi.evaluate(X_test_glove,y_test_glove)
print(history)
print("GLOVE_BI")

Epoch 1/20
125/125 [==============================] - 2s 13ms/step - loss: 1.2540 - accuracy: 0.4178 - val_loss: 1.1688 - val_accuracy: 0.4728
Epoch 2/20
125/125 [==============================] - 1s 9ms/step - loss: 1.0060 - accuracy: 0.5844 - val_loss: 0.9602 - val_accuracy: 0.5996
Epoch 3/20
125/125 [==============================] - 1s 9ms/step - loss: 0.8488 - accuracy: 0.6637 - val_loss: 0.9001 - val_accuracy: 0.6559
Epoch 4/20
125/125 [==============================] - 1s 9ms/step - loss: 0.7354 - accuracy: 0.7143 - val_loss: 0.8259 - val_accuracy: 0.6891
Epoch 5/20
125/125 [==============================] - 1s 9ms/step - loss: 0.6295 - accuracy: 0.7579 - val_loss: 0.8027 - val_accuracy: 0.6952
Epoch 6/20
125/125 [==============================] - 1s 9ms/step - loss: 0.5462 - accuracy: 0.7979 - val_loss: 0.7528 - val_accuracy: 0.7153
Epoch 7/20
125/125 [==============================] - 1s 9ms/step - loss: 0.4549 - accuracy: 0.8351 - val_loss: 0.7760 - val_accuracy: 0.7193
Epoch

In [15]:
model_glove_rnn = tf.keras.Sequential()
model_glove_rnn.add(tf.keras.Input(shape=(Max_input_size_glove,100)))
model_glove_rnn.add(tf.keras.layers.SimpleRNN(50))
model_glove_rnn.add(tf.keras.layers.Dense(4, activation='softmax'))
model_glove_rnn.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_glove_rnn.fit(X_train_glove,y_train_glove,epochs=20,validation_split=0.2)
history = model_glove_rnn.evaluate(X_test_glove,y_test_glove)
print(history)
print("GLOVE_RNN")

Epoch 1/20
125/125 [==============================] - 4s 31ms/step - loss: 1.3767 - accuracy: 0.3149 - val_loss: 1.3898 - val_accuracy: 0.2978
Epoch 2/20
125/125 [==============================] - 4s 30ms/step - loss: 1.3684 - accuracy: 0.3315 - val_loss: 1.4037 - val_accuracy: 0.2978
Epoch 3/20
125/125 [==============================] - 4s 32ms/step - loss: 1.3321 - accuracy: 0.3650 - val_loss: 1.2930 - val_accuracy: 0.4044
Epoch 4/20
125/125 [==============================] - 4s 32ms/step - loss: 1.2577 - accuracy: 0.4259 - val_loss: 1.2820 - val_accuracy: 0.3863
Epoch 5/20
125/125 [==============================] - 4s 30ms/step - loss: 1.2410 - accuracy: 0.4342 - val_loss: 1.2716 - val_accuracy: 0.4135
Epoch 6/20
125/125 [==============================] - 4s 31ms/step - loss: 1.2125 - accuracy: 0.4641 - val_loss: 1.2624 - val_accuracy: 0.4054
Epoch 7/20
125/125 [==============================] - 4s 30ms/step - loss: 1.2079 - accuracy: 0.4526 - val_loss: 1.2718 - val_accuracy: 0.4054